In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv

In [16]:
AirBnB = pd.read_csv("../original_data/Listings.csv", engine='python', encoding='ISO-8859-1')

In [17]:
AirBnB.head()

,listing_id,name,host_id,host_since,host_location,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_total_listings_count,...,minimum_nights,maximum_nights,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable
0,281420,"Beautiful Flat in le Village Montmartre, Paris",1466919,2011-12-03,"Paris, Ile-de-France, France",NaN,NaN,NaN,f,1.0,...,2,1125,100.0,10.0,10.0,10.0,10.0,10.0,10.0,f
1,3705183,39 mÃÂ² Paris (Sacre CÃ âur),10328771,2013-11-29,"Paris, Ile-de-France, France",NaN,NaN,NaN,f,1.0,...,2,1125,100.0,10.0,10.0,10.0,10.0,10.0,10.0,f
2,4082273,"Lovely apartment with Terrace, 60m2",19252768,2014-07-31,"Paris, Ile-de-France, France",NaN,NaN,NaN,f,1.0,...,2,1125,100.0,10.0,10.0,10.0,10.0,10.0,10.0,f
3,4797344,Cosy studio (close to Eiffel tower),10668311,2013-12-17,"Paris, Ile-de-France, France",NaN,NaN,NaN,f,1.0,...,2,1125,100.0,10.0,10.0,10.0,10.0,10.0,10.0,f
4,4823489,Close to Eiffel Tower - Beautiful flat : 2 rooms,24837558,2014-12-14,"Paris, Ile-de-France, France",NaN,NaN,NaN,f,1.0,...,2,1125,100.0,10.0,10.0,10.0,10.0,10.0,10.0,f


In [18]:
AirBnB.shape

(279712, 33)

In [19]:
AirBnB.describe()

,listing_id,host_id,host_response_rate,host_acceptance_rate,host_total_listings_count,latitude,longitude,accommodates,bedrooms,price,minimum_nights,maximum_nights,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value
count,2.797120e+05,2.797120e+05,150930.000000,166625.000000,279547.000000,279712.000000,279712.000000,279712.000000,250277.000000,279712.000000,279712.000000,2.797120e+05,188307.000000,187999.000000,188047.000000,187941.000000,188025.000000,187937.000000,187927.000000
mean,2.638196e+07,1.081658e+08,0.865939,0.827168,24.581612,18.761862,12.595075,3.288736,1.515509,608.792737,8.050967,2.755860e+04,93.405195,9.565476,9.312869,9.701534,9.698593,9.633994,9.335364
std,1.442576e+07,1.108570e+08,0.283744,0.289202,284.041143,32.560343,73.081309,2.133379,1.153080,3441.826611,31.518946,7.282875e+06,10.070437,0.990878,1.146072,0.867434,0.886884,0.833234,1.042625
min,2.577000e+03,1.822000e+03,0.000000,0.000000,0.000000,-34.264400,-99.339630,0.000000,1.000000,0.000000,1.000000,1.000000e+00,20.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000
25%,1.384462e+07,1.720656e+07,0.900000,0.780000,1.000000,-22.964390,-43.198040,2.000000,1.000000,75.000000,1.000000,4.500000e+01,91.000000,9.000000,9.000000,10.000000,10.000000,9.000000,9.000000
50%,2.767098e+07,5.826911e+07,1.000000,0.980000,1.000000,40.710785,2.382780,2.000000,1.000000,150.000000,2.000000,1.125000e+03,96.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000
75%,3.978485e+07,1.832853e+08,1.000000,1.000000,4.000000,41.908610,28.986730,4.000000,2.000000,474.000000,5.000000,1.125000e+03,100.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000
max,4.834353e+07,3.901874e+08,1.000000,1.000000,7235.000000,48.904910,151.339810,16.000000,50.000000,625216.000000,9999.000000,2.147484e+09,100.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000


In [20]:
AirBnB.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 279712 entries, 0 to 279711
Data columns (total 33 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   listing_id                   279712 non-null  int64  
 1   name                         279539 non-null  object 
 2   host_id                      279712 non-null  int64  
 3   host_since                   279547 non-null  object 
 4   host_location                278872 non-null  object 
 5   host_response_time           150930 non-null  object 
 6   host_response_rate           150930 non-null  float64
 7   host_acceptance_rate         166625 non-null  float64
 8   host_is_superhost            279547 non-null  object 
 9   host_total_listings_count    279547 non-null  float64
 10  host_has_profile_pic         279547 non-null  object 
 11  host_identity_verified       279547 non-null  object 
 12  neighbourhood                279712 non-null  object 
 13 

`price` is our target column that we will want to try and predict. The rest of the columns are potential features.

Now let's look at the number and % of missing values in our dataset.

In [21]:
missing = pd.concat([AirBnB.isnull().sum(), 100 * AirBnB.isnull().mean()], axis=1)
missing.columns=['count', '%']
missing.sort_values(by='count', ascending=False)

,count,%
district,242700,86.767818
host_response_time,128782,46.040928
host_response_rate,128782,46.040928
host_acceptance_rate,113087,40.429799
review_scores_value,91785,32.814109
review_scores_location,91775,32.810534
review_scores_checkin,91771,32.809104
review_scores_accuracy,91713,32.788368
review_scores_communication,91687,32.779073
review_scores_cleanliness,91665,32.771208


Looks like we are missing the majority (over 86%) of the `district` column. Since we have a `city`, a `neighbourhood` and coordinate columns (`latitude` and `longitude`), we should be able to safely drop the `district` column without losing much information. We are missing about a third of all the review related columns and over 40% and 46% of the `host_acceptance_rate` column and `host_response_rate` column (as well as `host_response_time` column) respectively. We will decide what we want to do with these columns/missing values in the future. We are not missing any of our target column, `price`, which is ideal.

In [22]:
AirBnB.district.value_counts()

Manhattan        16553
Brooklyn         14474
Queens            4704
Bronx              992
Staten Island      289
Name: district, dtype: int64

In [25]:
AirBnB.district.value_counts().sum()

37012

In [27]:
AirBnB[AirBnB['city'] == 'New York'].neighbourhood.value_counts()

Williamsburg          2733
Bedford-Stuyvesant    2711
Harlem                1972
Bushwick              1678
Hell's Kitchen        1506
                      ... 
Huguenot                 1
Woodrow                  1
Lighthouse Hill          1
Willowbrook              1
Castle Hill              1
Name: neighbourhood, Length: 220, dtype: int64

In [28]:
AirBnB[AirBnB['city'] == 'New York'].neighbourhood.value_counts().sum()

37012

With all of these districts being only in New York, we can definitely drop this column because it is not providing any extra information that isn't captured by the more specific `neighbourhood` column.

In [29]:
AirBnB = AirBnB.drop('district', axis=1)

Let's have a look at the rows of our dataframe and see if there are any duplicates.

In [34]:
#rows that are duplicated in certain columns (except for first instance) when not including listing_id 
dup_rows = AirBnB[AirBnB.drop('listing_id', axis=1).sort_values(by='').duplicated(['name', 'host_id', 'latitude', 'longitude', 'price'])]
dup_rows

,listing_id,name,host_id,host_since,host_location,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_total_listings_count,...,minimum_nights,maximum_nights,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable
45211,39192734,Louvre - Richelieu: cosy flat for 2,50978178,2015-12-10,FR,within a few hours,0.73,0.94,f,155.0,...,1,1125,NaN,NaN,NaN,NaN,NaN,NaN,NaN,f
45214,39211052,Louvre - Richelieu: cosy flat for 2,50978178,2015-12-10,FR,within a few hours,0.73,0.94,f,155.0,...,1,1125,50.0,6.0,9.0,8.0,6.0,9.0,5.0,f
45219,39221386,Louvre - Richelieu: cosy flat for 2,50978178,2015-12-10,FR,within a few hours,0.73,0.94,f,155.0,...,1,1125,72.0,8.0,9.0,8.0,8.0,10.0,8.0,f
53727,6473911,Central Apartment with 24/7 Concierge (1 guest),29087393,2015-03-10,"Rome, Lazio, Italy",NaN,NaN,NaN,f,9.0,...,1,1125,93.0,9.0,10.0,10.0,10.0,10.0,10.0,f
53728,6671646,Central Apartment with 24/7 Concierge (1 guest),29087393,2015-03-10,"Rome, Lazio, Italy",NaN,NaN,NaN,f,9.0,...,1,1125,80.0,8.0,9.0,9.0,9.0,8.0,8.0,f
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261629,40175560,The Dalebrook luxury sea-side apartment,30275500,2015-03-30,"Cape Town, Western Cape, South Africa",within a few hours,1.00,0.83,t,1.0,...,2,60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,f
261630,40175603,The Dalebrook luxury sea-side apartment,30275500,2015-03-30,"Cape Town, Western Cape, South Africa",within a few hours,1.00,0.83,t,1.0,...,2,60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,f
261631,40175641,The Dalebrook luxury sea-side apartment,30275500,2015-03-30,"Cape Town, Western Cape, South Africa",within a few hours,1.00,0.83,t,1.0,...,2,60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,f
263320,5832663,The Dalebrook luxury sea-side apartment,30275500,2015-03-30,"Cape Town, Western Cape, South Africa",within a few hours,1.00,0.83,t,1.0,...,2,60,100.0,10.0,10.0,10.0,10.0,10.0,10.0,f


In [14]:
#drop those rows by their index and then reset the index of the result
AirBnB = AirBnB.drop(index=dup_rows.index).reset_index()

Let's have a look at the Categorical features now.

In [32]:
AirBnB.select_dtypes('object')

,name,host_since,host_location,host_response_time,host_is_superhost,host_has_profile_pic,host_identity_verified,neighbourhood,city,property_type,room_type,amenities,instant_bookable
0,"Beautiful Flat in le Village Montmartre, Paris",2011-12-03,"Paris, Ile-de-France, France",NaN,f,t,f,Buttes-Montmartre,Paris,Entire apartment,Entire place,"[""Heating"", ""Kitchen"", ""Washer"", ""Wifi"", ""Long...",f
1,39 mÃÂ² Paris (Sacre CÃ âur),2013-11-29,"Paris, Ile-de-France, France",NaN,f,t,t,Buttes-Montmartre,Paris,Entire apartment,Entire place,"[""Shampoo"", ""Heating"", ""Kitchen"", ""Essentials""...",f
2,"Lovely apartment with Terrace, 60m2",2014-07-31,"Paris, Ile-de-France, France",NaN,f,t,f,Elysee,Paris,Entire apartment,Entire place,"[""Heating"", ""TV"", ""Kitchen"", ""Washer"", ""Wifi"",...",f
3,Cosy studio (close to Eiffel tower),2013-12-17,"Paris, Ile-de-France, France",NaN,f,t,t,Vaugirard,Paris,Entire apartment,Entire place,"[""Heating"", ""TV"", ""Kitchen"", ""Wifi"", ""Long ter...",f
4,Close to Eiffel Tower - Beautiful flat : 2 rooms,2014-12-14,"Paris, Ile-de-France, France",NaN,f,t,f,Passy,Paris,Entire apartment,Entire place,"[""Heating"", ""TV"", ""Kitchen"", ""Essentials"", ""Ha...",f
...,...,...,...,...,...,...,...,...,...,...,...,...,...
279707,Appartement T2 neuf prÃÂ¨s du tram T3a Porte ...,2015-04-13,"Paris, Ile-de-France, France",NaN,f,t,t,Observatoire,Paris,Entire apartment,Entire place,"[""Iron"", ""Heating"", ""Washer"", ""Dedicated works...",f
279708,Cozy Studio in Montmartre,2013-11-27,"Paris, Ile-de-France, France",NaN,f,t,t,Buttes-Montmartre,Paris,Entire apartment,Entire place,"[""Shampoo"", ""Iron"", ""Heating"", ""Washer"", ""Hair...",f
279709,Nice and cosy mini-appartement in Paris,2012-04-27,"Paris, Ile-de-France, France",NaN,f,t,t,Buttes-Montmartre,Paris,Entire apartment,Entire place,"[""Paid parking off premises"", ""Shampoo"", ""Firs...",f
279710,Charming apartment near Rue Saint Maur / Oberk...,2015-07-16,"Paris, Ile-de-France, France",NaN,f,t,t,Popincourt,Paris,Entire apartment,Entire place,"[""TV"", ""Iron"", ""Kitchen"", ""Hangers"", ""Smoke al...",f
